# Creating a Funding Transaction

In this section we'll create a Lightning Channel funding transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and the message the peers exchange to send and get the needed information from each other to make it happen. We'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook:
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'

## Channel Establishment v1

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/00-introduction.md)) defines two pathways to create a channel, lets focus on the legacy one because most of the channels are created using it, and lets the 'Channel Establishment v2' for the following notebook.

    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+

    - where node Alice is 'funder' and node Bob is 'fundee'

Alice send the [`open_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-open_channel-message) message to Bob. This message define many channel operational criterias, here we are going to focus on the information that are used in the funding transaction:
* chain_hash - denotes the exact blockchain that the opened channel will reside within.
* funding_satoshis - is the amount the sender is putting into the channel
* funding_pubkey - is the public key in the 2-of-2 multisig script of the funding transaction output.

If Bob does not agree with the criterias sent into the `open_channel` message he send a [`error_message`](https://github.com/lightning/bolts/blob/master/01-messaging.md#the-error-and-warning-messages) back to Alice, otherwise he sends the [`accept_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-accept_channel-message) message, containing some operational criterias from your side, here we are going to focus on the information that are used in the funding transaction:
* funding_pubkey - is the public key in the 2-of-2 multisig script of the funding transaction output.

## The Funding Transaction

The funding transaction input could be one or more UTXOs from Alice (the funder) and the outputs are: the change to Alice and a P2WSH output (pay-to-witness-script-hash)<sup>[BIP141](https://github.com/bitcoin/bips/blob/master/bip-0141.mediawiki#witness-program)</sup> funding the channel, the order of the pubkeys on the output script is defined lexicographically, where pubkey1 is the lesser of the two funding_pubkey in compressed format and pubkey2 is the greater.

`2 <pubkey1> <pubkey2> 2 OP_CHECKMULTISIG`

## Setup 

### Requirements
For this chapter we'll need the Bitcoin Core. This notebook has been tested with [v28.0](https://github.com/bitcoin/bitcoin/releases/tag/v28.0).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing lightning-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/home/pins-dev/Projects/lightning-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the lightning-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *

import json

In [3]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 0 - Key Derivation/The Lightning Node Keys.ipynb"

Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd973c123ac2dfd7ff
Bob Revocation Basepoint Private Key: fa526e09b9ff7425552db1803d7e12cf4475d060db4cfb812adb07219a54baa0
Bob Revocation Basepoint Public Key: 02536407aada57068f30b53c

### Creating the Inputs of the Channel Funding Transaction

In order to have Alice as funder of the channel we are going to create P2WPKH UTXO, to do that, we'll use the keys generated for Alice node into the previous notebook and convert them to p2wpkh addresses and send 0.201 BTC to Alice funding address, this transaction will be the input transaction of the Channel Funding transaction.

In [5]:
alice_funding_address = pk_to_p2wpkh(bytes.fromhex(alice_funding_pubkey), network = "regtest")
print("Alice's funding p2wpkh address: " + alice_funding_address)
alice_change_address = pk_to_p2wpkh(bytes.fromhex(alice_change_pubkey), network = "regtest")
print("Alice's change p2wpkh address: " + alice_change_address)

node = setup_testshell()
txid_to_spend, index_to_spend = fund_address(node, alice_funding_address, 0.201)
print(f"txid_to_spent: {txid_to_spend}, {index_to_spend}")

Alice's funding p2wpkh address: bcrt1qt0vq5dj8qa720lfx3hhzyu9ygsng9js3g02lnl
Alice's change p2wpkh address: bcrt1qt6ax8rh20h6zts0dcqmzdy9kte3pds4lrzwl6s
2025-01-19T20:17:24.897000Z TestFramework (INFO): PRNG seed is: 3934714372535145441
2025-01-19T20:17:24.901000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_40py99mc
txid_to_spent: 0a77041fd6c1b322b6a8c74457f7c333fce148a9ab50412343339e48532b09bd, 0


### Creating the Outputs of the Channel Funding Transaction

In order to create the output of the channel funding transaction we are going to create the 2 of 2 multisig redeem script and convert it to a P2WSH address, the oder output is the change address of Alice who is funding the channel.

In [9]:
# Compare the public keys lexicographically
if alice_funding_pubkey < bob_funding_pubkey:
    print(f"Alice pubkey is lexicographically lesser than Bob pubkey:\n{alice_funding_pubkey} < {bob_funding_pubkey}")
    pubkey1 = alice_funding_pubkey
    pubkey2 = bob_funding_pubkey
elif alice_funding_pubkey > bob_funding_pubkey:
    print(f"Alice pubkey is lexicographically greater than Bob pubkey:\n{alice_funding_pubkey} > {bob_funding_pubkey}")
    pubkey2 = alice_funding_pubkey
    pubkey1 = bob_funding_pubkey
else:
    print("Error: the public keys are equal.")

# Redeem Script: 2 <pubkey1> <pubkey2> 2 OP_CHECKMULTISIG
# "21" is the length of a 33 bytes (compressed) pubkey in hex notation
redeemScript = bytes.fromhex(
    "52"           # requires 2 signatures
    + "21"         # First pubkey
    + pubkey1      
    + "21"         # Second pubkey
    + pubkey2
    + "52"
    + "ae")        # OP_CHECKMULTISIG

print(f"Redeem Script:\n{redeemScript.hex()}")

channel_address = script_to_p2wsh(redeemScript, "regtest")
print("channel_address: ",channel_address)

# Our outputs
# Output1 is the Alice Change P2WPKH
output1_value_sat = int(float("0.15") * 100000000)
pk_hash = hash160(bytes.fromhex(alice_change_pubkey))
# Output2 is the Channel P2WSH
# The channel capacity is 5.000.000 of satoshis
output2_value_sat = int(float("0.05") * 100000000)
script_hash = hashlib.sha256(redeemScript).digest()
# The Alice UTXO was 0.201, we are leaving 0.001 as the fee of the funding trransaction

Alice pubkey is lexicographically lesser than Bob pubkey:
02245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb963 < 032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c3
Redeem Script:
522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae
channel_address:  bcrt1qhsn37ekwjgxlajuavyzne28yrahkyesgnr6qhxvzlw3807ttcjeqhcl6mc


### Creating the Unsigned Channel Funding Transaction

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the witness field is empty.

In [10]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER
marker = bytes.fromhex("00")

# FLAG
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 to Alice Change Address
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' 
output1_spk = bytes.fromhex("0014") + pk_hash


# OUTPUT 2 to Channel Funding Address
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk'
output2_spk = bytes.fromhex("0020") + script_hash

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

print("unsigned_tx: ", unsigned_tx.hex())



unsigned_tx:  0200000001bd092b53489e3343234150aba948e1fc33c3f75744c7a8b622b3c1d61f04770a0000000000ffffffff02c0e1e400000000001600145eba638eea7df425c1edc0362690b65e6216c2bf404b4c0000000000220020bc271f66ce920dfecb9d61053ca8e41f6f62660898f40b9982fba277f96bc4b200000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

Just uncomment the lines below to do that.

In [11]:
# decoded = node.decoderawtransaction(unsigned_tx.hex())
# print(json.dumps(decoded, indent=2, default=str))

### Signing the Channel Funding Transaction

Segwit transactions have a signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki). According to the sighash type (ALL, NONE, SINGLE, ANYONECANPAY), a transaction digest is generated with a double SHA256 of a serialized subset of the transaction (tx_digest_preimage), and the signature is verified against this digest with a given public key.

#### Creating the Transaction Digest Preimage

In [12]:
pk_hash = hash160(bytes.fromhex(alice_funding_pubkey))
scriptcode = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

input_amount_sat = int(0.201 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print("tx_digest_preimage: ",tx_digest_preimage.hex())

tx_digest_preimage:  02000000b9076efe15da962420223f4f209f72153bc8660fa15d073eb3482fd5d3cdba6e3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044bd092b53489e3343234150aba948e1fc33c3f75744c7a8b622b3c1d61f04770a000000001976a9145bd80a3647077ca7fd268dee2270a4442682ca1188aca0b3320100000000fffffffff024a453c206730b975be44da5987693c127c508aada9aabadc9455fb74b5fc60000000001000000


#### Creating the Transaction Digest and Signing it

Now we are ready to hash this transaction digest preimage and produce an ecdsa signature on it.

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [13]:
# Create the Transaction Digest (sigHash) to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(bytes.fromhex(alice_funding_privkey), curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Witness field
witness = (
    # indicate the number of stack items for the txin
    # 2 items for signature and pubkey
    bytes.fromhex("02")
    + varint_len(signature)
    + signature
    + varint_len(bytes.fromhex(alice_funding_pubkey))
    + bytes.fromhex(alice_funding_pubkey)
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs # scriptsig left empty as espcified on BIP
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  02000000000101bd092b53489e3343234150aba948e1fc33c3f75744c7a8b622b3c1d61f04770a0000000000ffffffff02c0e1e400000000001600145eba638eea7df425c1edc0362690b65e6216c2bf404b4c0000000000220020bc271f66ce920dfecb9d61053ca8e41f6f62660898f40b9982fba277f96bc4b20247304402201bf5e30e5f67ec27c6d16bb694adb6c362b52f8223bf7748908ee7910d232ca802205c968d516430cbb3b58f57ee5ae7edd607b44bfe9e3e15d7904a80f556a58491012102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96300000000


### The funding_created Message
Now Alice can send Bob the [`funding_created`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-funding_created-message) message with the following information from the channel funding transaction:
- funding_txid
- funding_output_index
- signature

This message describes the outpoint which Alice has created for the initial commitment transactions.

Alice should never broadcast this channel funding transaction before having the Bob signature for the commitment transaction, otherwise she would have to trust Bob to get her funds back from the multisig UTXO. So let's got to the next [notebook](https://github.com/MPins/lightning-tx-tutorial/blob/master/Chapter%202%20-%20Commitment%20Transactions/Alice%20Commitment%20Transaction.ipynb) to see the commitment transactions construction in detail.

To see if our channel funding transaction would have been accepted, we can use the testmempoolaccept command below.


In [14]:
result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

[{'txid': 'b0a646de1ca1cd72a555fecd15acaa3fa30299ac2ba9c54d23ed058e012e361a', 'wtxid': 'ea72bf429d7f0cd71f3ef64c55ab3c33539702a6b82dafd14018443768d05695', 'allowed': True, 'vsize': 153, 'fees': {'base': Decimal('0.00100000'), 'effective-feerate': Decimal('0.00653594'), 'effective-includes': ['ea72bf429d7f0cd71f3ef64c55ab3c33539702a6b82dafd14018443768d05695']}}]
